In [1]:
import numpy as np
import pulp as pl

pl.list_solvers(onlyAvailable=True)

['GLPK_CMD', 'GUROBI_CMD', 'PULP_CBC_CMD', 'PULP_CHOCO_CMD']

In [2]:
model = pl.LpProblem(name="quiz-1", sense=pl.LpMaximize)

x_vars = {}
for i in range(1, 3):
    x_vars[i] = pl.LpVariable(name=f"x_{i}", lowBound=0, cat="Continuous")

obj_func = x_vars[1] + 3 * x_vars[2]
model += obj_func

# Constraints
model += (-1 * x_vars[1] + x_vars[2] <= 3, "constraint_1")
model += (-1 * x_vars[1] + 2 * x_vars[2] <= 8, "constraint_2")
model += (3 * x_vars[1] + x_vars[2] <= 18, "constraint_3")

status = model.solve(solver=pl.PULP_CBC_CMD(msg=0))
if status == 1:
    print(model.objective.value())
    for var in model.variables():
        print(var.name, "=", var.value())

22.0
x_1 = 4.0
x_2 = 6.0


In [3]:
# Solve manually